<a href="https://colab.research.google.com/github/XuanLoc2578/NLP_Basic/blob/main/Torch_NameClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import numpy as np

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import time
import math
import random

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
import unicodedata
import string

In [ ]:
files = glob.glob('/content/sample_data/*.txt', recursive = True)

In [ ]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [ ]:
all_letters, n_letters

("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'", 57)

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicode2Ascii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
      and c in all_letters
  )

print(unicode2Ascii('Nguyễn'))

Nguyen


In [ ]:
#Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

In [ ]:
# Read a file and split into lines
def readLines(filename):
  lines = open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicode2Ascii(line) for line in lines]

In [ ]:
for filename in files:
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readLines(filename)
  category_lines[category] = lines

n_categories = len(all_categories)

In [ ]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [ ]:
def letterToIndex(letter):
  return all_letters.find(letter)

In [ ]:
def letterToTensor(letter):
  tensor = torch.zeros(1, n_letters)
  tensor[0][letterToIndex(letter)] = 1
  return tensor

In [ ]:
def lineToTensor(line):
  tensor = torch.zeros(len(line), 1, n_letters)
  for li, letter in enumerate(line):
    tensor[li][0][letterToIndex(letter)] = 1
  return tensor

In [ ]:
letterToIndex('J'), letterToTensor('J'), lineToTensor('Jones').shape

(35,
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0.]]),
 torch.Size([5, 1, 57]))

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
model = RNN(n_letters, n_hidden, n_categories)

In [ ]:
n_letters, n_hidden, n_categories

(57, 128, 18)

In [ ]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = model(input, hidden)

In [ ]:
input.shape, hidden.shape, output.shape

(torch.Size([1, 57]), torch.Size([1, 128]), torch.Size([1, 18]))

In [ ]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = model(input[0], hidden)
print(output, output.shape)

tensor([[-2.9304, -2.7751, -2.8999, -2.8807, -2.9345, -2.8431, -2.8254, -2.9800,
         -2.8412, -2.8677, -2.8996, -2.9297, -2.9592, -2.8788, -2.9226, -2.7675,
         -2.9281, -2.9985]], grad_fn=<LogSoftmaxBackward0>) torch.Size([1, 18])


In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    # print(top_n, top_i)
    category_i = top_i.item()
    # print(top_i.item())
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('French', 15)


In [ ]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = English / line = Wiles
category = Japanese / line = Hishida
category = English / line = Mullen
category = Arabic / line = Sleiman
category = French / line = Cornett
category = Polish / line = Szewc
category = English / line = Hetherington
category = Scottish / line = Milne
category = German / line = Herrmann
category = German / line = Lohrenz


In [ ]:
lr = 0.005

criterion = nn.NLLLoss()
solver = optim.SGD(model.parameters(), lr=lr)

In [ ]:
def train(category_tensor, line_tensor):
    hidden = model.initHidden()

    model.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = model(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    # for p in model.parameters():
    #     p.data.add_(p.grad.data, alpha=-learning_rate)
    solver.step()

    return output, loss.item()

In [ ]:
line_tensor[0]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])

In [ ]:
n_iters = 85000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 5s) 1.3360 Baba / Arabic ✗ (Japanese)
10000 11% (0m 11s) 2.9626 Apeldoorn / Italian ✗ (Dutch)
15000 17% (0m 16s) nan Bishara / Vietnamese ✗ (Arabic)
20000 23% (0m 22s) nan Koolen / Vietnamese ✗ (Dutch)
25000 29% (0m 27s) nan Brant / Vietnamese ✗ (German)
30000 35% (0m 33s) nan Kruger / Vietnamese ✗ (German)
35000 41% (0m 39s) nan Maclean / Vietnamese ✗ (Scottish)
40000 47% (0m 44s) nan Isa / Vietnamese ✗ (Arabic)
45000 52% (0m 50s) nan Fiscella / Vietnamese ✗ (Italian)
50000 58% (0m 55s) nan Sturm / Vietnamese ✗ (German)
55000 64% (1m 1s) nan Hodowal / Vietnamese ✗ (Czech)
60000 70% (1m 6s) nan Burns / Vietnamese ✗ (Scottish)


KeyboardInterrupt: ignored

In [ ]:
plt.figure()
plt.plot(all_losses)

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = model.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = model(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, sorted=True)
        # predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            # predictions.append([value, all_categories[category_index]])

In [ ]:
predict('Pirlo')
predict('Pirli')
predict('Pirlioni')